In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import numpy as np
import pandas as pd
import xarray as xr

from utils import AHI_BANDS, ABI_BANDS, remap, WMO_IDS

from tqdm import tqdm

import netCDF4

from pathlib import Path

In [32]:
SATZEN_CACHE = Path('satzen_cache/')
INDEX = Path('index')
COMP_CACHE = Path('composite_cache')
COMP_CACHE.mkdir(exist_ok=True)

In [33]:
grid_shape = int(180/.05), int(360/.05)

In [39]:
satzens = xr.DataArray(np.full((3, *grid_shape), np.nan, dtype=np.float32), dims=['layer','latitude','longitude'])
wmo_ids = xr.DataArray(np.full((3, *grid_shape), np.nan, dtype=np.uint16), dims=['layer','latitude','longitude'])
with tqdm([('g16',ABI_BANDS),('g17', ABI_BANDS),('h8', AHI_BANDS)]) as bar:
    for i,(sat, band_lookup) in enumerate(bar):
        wmo_id = WMO_IDS[sat]
        index_band = band_lookup['temp_11_00um']
        index_dir = INDEX / sat / f'{index_band:02d}'
        src_index = np.memmap(index_dir / 'src_index.dat', mode='r', dtype=np.uint64)
        dst_index = np.memmap(index_dir / 'dst_index.dat', mode='r', dtype=np.uint64)
        satzen = xr.open_dataset(SATZEN_CACHE / f'{sat}_satzen.nc')
        satzens.values[i,...] = remap(src_index, dst_index, satzen.satzen.values, grid_shape)
        wmo_ids[i,...] = wmo_id

100%|██████████| 3/3 [00:06<00:00,  2.01s/it]


In [40]:
argsort = satzens.fillna(900).argsort(axis=0)

In [41]:
satzens_sort = xr.DataArray(np.take_along_axis(satzens.values, argsort, 0), dims=satzens.dims)
wmo_ids_sort = xr.DataArray(np.take_along_axis(wmo_ids.values, argsort, 0), dims=satids.dims)

In [44]:
satids_sort.to_dataset(name='wmo_id').to_netcdf(COMP_CACHE / 'wmo_id.nc', encoding={'wmo_id':{'zlib':True}})

In [46]:
satzens_sort.to_dataset(name='satzen').to_netcdf(COMP_CACHE / 'satzen.nc',
                                                 encoding={
                                                     'satzen':{'zlib':True, 'scale_factor':.1,
                                                               'dtype':'i2',
                                                               '_FillValue':netCDF4.default_fillvals['i2']}})

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import get_sorting

In [6]:
get_sorting.main()

100%|██████████| 3/3 [00:05<00:00,  2.00s/it]
